In [75]:
import os

#large bounding box for final analysis
#bbox = [-73.125276,40.845996,-61.913572,46.149860]

#small bounding box for testing
bbox = [-70.51364, 41.47909, -69.30295, 43.438689999999994]

lonlat_span = [  abs(bbox[0]-bbox[2]) , abs(bbox[1]-bbox[3])  ]
print( lonlat_span )


[1.2106899999999996, 1.9595999999999947]


In [ ]:
#make temp lookup



In [78]:

def get_bth( obj ):
    try:
        
        test = subprocess.Popen(["gdallocationinfo",
                            "-valonly",
                            "-wgs84",
                            "../data/raw/bathymetry/tows-area/bathymetry.tif",
                            obj[0],
                            obj[1]
                            ], stdout=subprocess.PIPE)  
        output = test.communicate()
        bth = float(output[0].decode("utf-8"))
        return bth if bth <= 0 else None
    except:
        return None

import subprocess   


def get_sst( obj ):
    
    sample_sst_file = '../data/raw/aquaMODIS-L2-sst-avg/avgsst/06sstavg.tiff'
    
    test = subprocess.Popen(["gdallocationinfo",
                            "-valonly",
                            "-wgs84",
                            sample_sst_file,
                            str(obj[0]),
                            str(obj[1])
                            ], stdout=subprocess.PIPE)  
    output = test.communicate()
    val = output[0].decode("utf-8")
    #cel = ((val / 255) * 47) -2
    
    print( val )
    
    return 

get_raster_index( -70.41364, 41.47909 )

def get_raster_index( lon, lat ):
    base_file = '../data/raw/aquaMODIS-L2-sst-avg/avgsst/baseeval.tif'
    test = subprocess.Popen(["gdallocationinfo",
                            "-xml",
                            "-wgs84",
                            base_file,
                            str(lon),
                            str(lat)
                            ], stdout=subprocess.PIPE)  
    output = test.communicate()
    
    val = output[0].decode("utf-8")
    #cel = ((val / 255) * 47) -2
    print(output, val  )
    
    return 

#get_raster_index( -73.025,40.94 )


(('<Report pixel="324" line="560">\n  <BandReport band="1">\n    <Value>-13</Value>\n  </BandReport>\n</Report>\n', None), u'<Report pixel="324" line="560">\n  <BandReport band="1">\n    <Value>-13</Value>\n  </BandReport>\n</Report>\n')


In [79]:
ndiv = 10

records = []

for lon_index in range( ndiv ):
    
    left = bbox[0] + ( (lonlat_span[0]/ float(ndiv)) * lon_index )
    right = left + (lonlat_span[0]/ float(ndiv))
    
    for lat_index in range( ndiv ):
        #bathymetry info for grid cell
        bottom = bbox[1] + ( (lonlat_span[1]/ float(ndiv)) * lat_index )
        top = bottom + (lonlat_span[1]/ float(ndiv))
        grid_dict = { 'left':left, 'bottom': bottom, 'right':right, 'top':top  }
        records.append( [left,bottom,right,top]  ) 
        bth = get_bth([(left+right/2), (top+bottom)/2])
        if bth:    
            print( bth )
            
        #ocean temp for grid cell
        get_sst( [(left+right)/2, (top+bottom)/2] )
        
        
        #get cells of base raster to paint
        
        #print( left, bottom, top, right )
        
#         get_raster_index( left, top )
#         get_raster_index( right, top )
#         get_raster_index( right, bottom )
#         get_raster_index( left, bottom )
        
        
        
        
# print( records[:10] )
# print( len(records) )


51

51

51

41

41

41

41

41

84

84

51

51

51

41

41

41

41

41

84

84

51

51

51

41

41

41

41

41

84

84

51

51

51

41

41

41

41

41

84

84

20

20

20

39

39

39

39

39

35

35

20

20

20

39

39

39

39

39

35

35

20

20

20

39

39

39

39

39

35

35

20

20

20

39

39

39

39

39

35

35

20

20

20

39

39

39

39

39

35

35

20

20

20

39

39

39

39

39

35

35



In [59]:
import rasterio
import numpy as np

base_file = '../raw/aquaMODIS-L2-sst-avg/avgsst/GEBCO2014_-73.12_40.84_-61.91_46.149_30Sec_Geotiff.tif'

with rasterio.open('../data/raw/bathymetry/eval-area/GEBCO2014_-73.12_40.84_-61.91_46.149_30Sec_Geotiff.tif') as src:
    print( src.shape, src.count )
    
    
    base_array = np.zeros( src.shape )
    #o = src.read()
    #print(len(o[0][0]))
    
    print( base_array.shape )



((637, 1345), 1)
(637, 1345)


In [ ]:
import rasterio
from rasterio.transform import from_origin

arr = np.random.randint(5, size=(100,100)).astype(np.float)

transform = from_origin(472137, 5015782, 0.5, 0.5)

new_dataset = rasterio.open('test1.tif', 'w', driver='GTiff',
                            height = arr.shape[0], width = arr.shape[1],
                            count=1, dtype=str(arr.dtype),
                            crs='+proj=utm +zone=10 +ellps=GRS80 +datum=NAD83 +units=m +no_defs',
                            transform=transform)

new_dataset.write(arr, 1)
new_dataset.close()



